In [ ]:
from bwb import logging
logging.log_config.set_level(logging.)

In [2]:

import time

import torch
from torch import linalg as LA

from bwb.distributions import *
from bwb.distributions.data_loaders import *
from bwb.transports import *

In [3]:
arr = np.load(r"..\data\face.npy")
arr.shape

(161666, 784)

In [4]:
faces = DistributionDrawDataLoader(arr, (28, 28))
faces

DistributionDrawDataLoader((n_models, n_supp)=(161666, 784))

In [5]:
first_face = faces[0]
first_face

In [6]:
x = first_face.sample((10,))
x

tensor([623, 691, 592, 338, 688, 355, 584, 601, 304, 260])

In [9]:
%%time
pi_n: ExplicitPosteriorPiN[DistributionDraw] = ExplicitPosteriorPiN()
pi_n

CPU times: total: 0 ns
Wall time: 0 ns


ExplicitPosteriorPiN()

In [10]:
%%time
pi_n.fit(data=x, models=faces)

CPU times: total: 250 ms
Wall time: 122 ms


ExplicitPosteriorPiN(n_data=10, n_models=161666)

In [11]:
def gamma(t, a=100, b=110, c=0.51):
    return a / (b ** (1/c) + t) ** c

for t in range(10):
    print(f"{t = }; {gamma(t) = :.8f}")
print()

for t in range(90, 100):
    print(f"{t = }; {gamma(t) = :.8f}")
print()

for t in range(1000, 1010):
    print(f"{t = }; {gamma(t) = :.8f}")
print()

for t in range(3000, 3010):
    print(f"{t = }; {gamma(t) = :.8f}")

t = 0; gamma(t) = 0.90909091
t = 1; gamma(t) = 0.90904484
t = 2; gamma(t) = 0.90899878
t = 3; gamma(t) = 0.90895272
t = 4; gamma(t) = 0.90890667
t = 5; gamma(t) = 0.90886063
t = 6; gamma(t) = 0.90881460
t = 7; gamma(t) = 0.90876857
t = 8; gamma(t) = 0.90872255
t = 9; gamma(t) = 0.90867653

t = 90; gamma(t) = 0.90497213
t = 91; gamma(t) = 0.90492667
t = 92; gamma(t) = 0.90488123
t = 93; gamma(t) = 0.90483579
t = 94; gamma(t) = 0.90479035
t = 95; gamma(t) = 0.90474492
t = 96; gamma(t) = 0.90469950
t = 97; gamma(t) = 0.90465409
t = 98; gamma(t) = 0.90460868
t = 99; gamma(t) = 0.90456328

t = 1000; gamma(t) = 0.86621020
t = 1001; gamma(t) = 0.86617027
t = 1002; gamma(t) = 0.86613034
t = 1003; gamma(t) = 0.86609043
t = 1004; gamma(t) = 0.86605051
t = 1005; gamma(t) = 0.86601061
t = 1006; gamma(t) = 0.86597070
t = 1007; gamma(t) = 0.86593081
t = 1008; gamma(t) = 0.86589092
t = 1009; gamma(t) = 0.86585103

t = 3000; gamma(t) = 0.79582361
t = 3001; gamma(t) = 0.79579254
t = 3002; gamma(t) = 0.

In [66]:
%%time
eps = 0.
alpha = 1./10
transp = EMDTransport()
max_time_iter = 10 # 60 * 60  # Una hora
max_iter = float("inf")

# Camino de baricentros
position_historial = []

# Paso 1: samplear un mu_0
mu_0 = pi_n.draw()

# Calcular las masas y ubicaciones
X, m_ = [], []
min_w, max_w = mu_0.non_zero_probs.min(), mu_0.non_zero_probs.max()

for x, w, n in zip(mu_0.enumerate_non_zero_support_(), mu_0.non_zero_probs,
                   torch.ceil(alpha * mu_0.non_zero_probs / min_w).to(int)):
    for _ in range(n):
        X.append(x.reshape(1, -1))
        m_.append(w / n)

X_k = torch.cat(X, 0)
m = torch.as_tensor(m_, dtype=mu_0.dtype, device=mu_0.device)
position_historial.append(X_k)

print(f"{len(X_k) = }")

k = 0

tic, toc = time.time(), time.time()

while k < max_iter and toc - tic < max_time_iter:
    print("\n" + "="*10 + f" {k = }, Δt = {toc - tic:.4f} [seg] " + "="*10)
    # Paso 2: Samplear \tilde\mu_k
    t_mu_k: DistributionDraw = pi_n.draw()
    t_X_k, t_m_k = t_mu_k.enumerate_non_zero_support_(), t_mu_k.non_zero_probs

    # Calcular transporte óptimo
    transp.fit_wm(
        Xs=X_k, mu_s=m,
        Xt=t_X_k, mu_t=t_m_k,
    )
    T_X_k = transp.transform(X_k)
    print(f"{T_X_k.shape = }")

    # Calcular la distribución de mu_{k+1}
    gamma_k = gamma(k); print(f"{gamma_k = :.6f}")
    X_kp1 = (1 - gamma_k) * X_k + gamma_k * T_X_k  # Basta con calcular las transformaciones, porque los pesos son los mismos
    print(f"{X_kp1.shape = }")

    # Calcular la distancia de Wasserstein
    diff = X_k - T_X_k
    dist = float((gamma_k ** 2) * torch.sum(m * LA.norm(diff, dim=1) ** 2))
    print(f"{dist = :.8f}")

    while dist < eps:
        break

    position_historial.append(X_kp1)

    k += 1
    X_k = X_kp1
    toc = time.time()

len(X_k) = 5051

========== k = 0, Δt = 0.0000 [seg] ==========
T_X_k.shape = torch.Size([5051, 2])
gamma_k = 0.894427
X_kp1.shape = torch.Size([5051, 2])
dist = 7.04906251

========== k = 1, Δt = 1.5431 [seg] ==========
T_X_k.shape = torch.Size([5051, 2])
gamma_k = 0.868500
X_kp1.shape = torch.Size([5051, 2])
dist = 2.65583110

========== k = 2, Δt = 2.3322 [seg] ==========
T_X_k.shape = torch.Size([5051, 2])
gamma_k = 0.844262
X_kp1.shape = torch.Size([5051, 2])
dist = 2.00642268

========== k = 3, Δt = 2.9719 [seg] ==========
T_X_k.shape = torch.Size([5051, 2])
gamma_k = 0.821545
X_kp1.shape = torch.Size([5051, 2])
dist = 1.54090326

========== k = 4, Δt = 3.5793 [seg] ==========
T_X_k.shape = torch.Size([5051, 2])
gamma_k = 0.800205
X_kp1.shape = torch.Size([5051, 2])
dist = 3.44264262

========== k = 5, Δt = 4.1066 [seg] ==========
T_X_k.shape = torch.Size([5051, 2])
gamma_k = 0.780116
X_kp1.shape = torch.Size([5051, 2])
dist = 2.60890346

========== k = 6, Δt = 4.5827 [seg] =====

In [93]:
%%time
reshaped_positions = []
n_coord, n_dim = position_historial[0].shape
for p in position_historial:
    p_reshape = p.reshape(n_coord, 1, n_dim)
    reshaped_positions.append(p_reshape)

CPU times: total: 0 ns
Wall time: 894 µs


In [94]:
%%time
positions_historial_batch = torch.cat(reshaped_positions, dim=1)

CPU times: total: 46.9 ms
Wall time: 7.95 ms


In [95]:
positions_historial_batch.shape

torch.Size([5051, 13, 2])

In [3]:
import pickle
from pathlib import Path
import os


SAVE = False


PICKLES_PATH = Path("./pickles")

POSITION_HISTORIAL_PATH = PICKLES_PATH / "postion_historial.dat"
WEIGHTS_PATH = PICKLES_PATH / "weights.dat"

if not PICKLES_PATH.exists(): PICKLES_PATH.mkdir()

if SAVE:
    if POSITION_HISTORIAL_PATH.exists():
        os.remove(POSITION_HISTORIAL_PATH)

    if WEIGHTS_PATH.exists():
        os.remove(WEIGHTS_PATH)

    with POSITION_HISTORIAL_PATH.open("wb") as f:
        pickle.dump(positions_historial_batch, f)

    with WEIGHTS_PATH.open("wb") as f:
        pickle.dump(m, f)

PICKLES_PATH.exists()

True

In [2]:
import pickle
from pathlib import Path

d = 2
PICKLES_PATH = Path("./pickles")
POSITION_HISTORIAL_PATH = PICKLES_PATH / f"postion_historial{d}.dat"
WEIGHTS_PATH = PICKLES_PATH / f"weights{d}.dat"

with POSITION_HISTORIAL_PATH.open("rb") as f:
    new_position_historial = pickle.load(f)

with WEIGHTS_PATH.open("rb") as f:
    m = pickle.load(f)

new_position_historial, m

([tensor([[ 2., 10.],
          [ 2., 11.],
          [ 2., 12.],
          [ 2., 13.],
          [ 2., 14.],
          [ 2., 15.],
          [ 2., 16.],
          [ 2., 17.],
          [ 3.,  7.],
          [ 3.,  8.],
          [ 3.,  9.],
          [ 3., 10.],
          [ 3., 11.],
          [ 3., 12.],
          [ 3., 13.],
          [ 3., 14.],
          [ 3., 15.],
          [ 3., 16.],
          [ 3., 17.],
          [ 4.,  5.],
          [ 4.,  6.],
          [ 4.,  7.],
          [ 4.,  8.],
          [ 4.,  9.],
          [ 4., 10.],
          [ 4., 11.],
          [ 4., 12.],
          [ 4., 13.],
          [ 4., 14.],
          [ 4., 15.],
          [ 4., 16.],
          [ 4., 17.],
          [ 4., 18.],
          [ 4., 19.],
          [ 5.,  3.],
          [ 5.,  4.],
          [ 5.,  5.],
          [ 5.,  6.],
          [ 5.,  7.],
          [ 5.,  8.],
          [ 5.,  9.],
          [ 5., 15.],
          [ 5., 16.],
          [ 5., 17.],
          [ 5., 18.],
          

In [5]:
len(new_position_historial)

6361

In [5]:
new_position_historial[0].shape, new_position_historial[1].shape, new_position_historial[2].shape

(torch.Size([307, 2]), torch.Size([4357, 2]), torch.Size([4357, 2]))

In [6]:
new_position_historial = new_position_historial[1:]

In [7]:
%%time
reshaped_positions = []
n_coord, n_dim = new_position_historial[0].shape
for p in new_position_historial:
    p_reshape = p.reshape(n_coord, 1, n_dim)
    reshaped_positions.append(p_reshape)

CPU times: total: 46.9 ms
Wall time: 46.9 ms


In [8]:
%%time
positions_historial_batch = torch.cat(reshaped_positions, dim=1)

CPU times: total: 2.25 s
Wall time: 2.96 s


In [9]:
positions_historial_batch.shape

torch.Size([4357, 6360, 2])

In [10]:
m.shape

torch.Size([4357])

In [11]:
# position_historial = new_position_historial
# position_historial[:, 0, :].shape

In [12]:
# pos_hist_rounded = torch.round(positions_historial_batch)
# pos_hist_ind = (pos_hist_rounded[:, :, 0] * m + pos_hist_rounded[:, :, 1]).to(int)
# pos_hist_ind

In [13]:
# pos_hist_ind.shape

In [14]:
# m.shape

In [15]:
# to_return = torch.zeros((13, 28*28))
#
# for w, p in zip(m, pos_hist_ind):
#     for i, pp in enumerate(p):
#         to_return[i, pp] += w

In [16]:
# to_return.sum(1)

In [28]:
%%time
def position_to_weights(positions, weights, shape):

    positions, weights = torch.as_tensor(positions), torch.as_tensor(weights)

    n, m  = shape
    n_coord, n_pos, n_dim = positions.shape

    to_return = torch.zeros((n_pos, n * m))
    pos_rounded = torch.round(positions)
    pos_ind = (pos_rounded[:, :, 0]*m + pos_rounded[:, :, 1]).to(int)

    for w, p in zip(weights, pos_ind):
        for i, pp in enumerate(p):
            to_return[i, pp] += w

    return to_return

# ptw = position_to_weights([[0, 0], [0, 0], [1, 0], [1, 1]], [0.2, 0.3, 0.1, 0.4], (2, 2))
# ptw

ptw = position_to_weights(positions_historial_batch[:, :10, :], m, (28, 28))

CPU times: total: 2.3 s
Wall time: 2.66 s


In [31]:
def position_to_list_of_dd(positions, weights, shape, i, j):
    return [DistributionDraw(w, shape) for w in position_to_weights(positions[:, i:j, :], weights, shape)]
list_dd = position_to_list_of_dd(positions_historial_batch[:, :10, :], m, (28, 28), 0, 10)
list_dd

[DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28)),
 DistributionDraw(shape: (28, 28))]

In [37]:
from bwb.utils import plot_list_of_draws
list_dd = position_to_list_of_dd(positions_historial_batch, m, (28, 28), 0, 36)
plot_list_of_draws(list_dd)

In [39]:
list_dd = position_to_list_of_dd(positions_historial_batch, m, (28, 28), 36, 72)
plot_list_of_draws(list_dd)

In [38]:
list_dd = position_to_list_of_dd(positions_historial_batch, m, (28, 28), -37, -1)
plot_list_of_draws(list_dd)

In [20]:
n_coord, n_pos, n_dim = positions_historial_batch.shape
to_return = torch.zeros((n_pos, 28*28))
pos_rounded = torch.round(positions_historial_batch)
pos_ind = (pos_rounded[:, :, 0]*28 + pos_rounded[:, :, 1]).to(int)

gen = zip(m, pos_ind)
w, p = next(gen)
w.shape, p.shape

(torch.Size([]), torch.Size([6360]))

In [26]:
to_return[:, 0].shape

torch.Size([6360])

In [ ]:
ptw

In [ ]:
%%time
historial_bsgd = torch.concat([position_to_weights(p, m, (28, 28)).reshape((1, -1)) for p in new_position_historial], 0)

In [ ]:
dddl = DistributionDrawDataLoader(

)

In [ ]:
arr = torch.arange(28*28)
# arr = torch.reshape(arr, (28, 28))

In [36]:
pos = torch.round(new_position_historial[-1][:10]).to(int)
pos_ind = pos[:, 0] * 28 + pos[:, 1]
# for p in map(tuple, pos):
#     print(arr[p])
# pos_ind

arr[pos_ind]

tensor([93, 93, 92, 94, 94, 94, 93, 93, 95, 95])